In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.01
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [3]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0., .05, 0.]
u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# q0 = [1.0,7.14e-12,7.14e-12,7.14e-12,0.016266,7.52931e-12,-3.01284e-9]
# v0 = [7.0e-12,7.0e-12,7.0e-12,0.126601,6.52901e-12,-0.439901]
# u0 = zeros(sim_data.num_v)
# qnext = [1.0,8.18e-12,8.18e-12,8.18e-12,0.016266,8.52931e-12,-4.08867e-9 ]
# vnext = [ 8.0e-12,8.0e-12,8.0e-12,9.05748e-9,1.42951e-18,-1.07683e-7 ]
# x_sol_exp = [0.302637,0.214285,1.59316,0.214285,8.04123e-9,5.48421]

z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))

num_steps_default = 10
α_vect_default = [1.^i for i in 1:num_steps_default]
c_vect_default = [200.+min.(2.^i,500) for i in 1:num_steps_default]
I_vect_default = 1e-16*ones(num_steps_default)

τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=true);
τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=α_vect_default,c_vect=c_vect_default,I_vect=I_vect_default);

display("Explicit")
display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display(norm(x_sol_ip-x_sol_auglag))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded


"Explicit"

6-element Array{Float64,1}:
 3.89397e-8
 4.57664e-8
 3.89397e-8
 0.00999961
 0.0495097 
 0.999977  

"IP"

6-element Array{Float64,1}:
  2.04278e-7
 -1.08862e-8
  2.04278e-7
  0.00999936
  0.0495095 
  0.999977  

"Aug Lag"

6-element Array{Float64,1}:
 0.00173863
 0.00124545
 0.00173863
 0.0032352 
 0.0296354 
 0.971562  

0.035436424354566835

Solve_Succeeded


In [ ]:
ForwardDiff.jacobian(x̃ -> solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,x̃[1:sim_data.num_q],x̃[sim_data.num_q+1:sim_data.num_q+sim_data.num_v];ip_method=false)[1],vcat(qnext,vnext))